# F. MPCA Vega Hedge


## Data Decomposition
In curve PCA, we put together all daily curve movement (row) vectors from the past 3 months and form a data matrix $X$. We then perform SVD to obtain 
$$
X = TV^{\mathsf T}, 
$$
where $T$ is the PC score matrix, and $V^{\mathsf T}$ is a unitary matrix whose rows are the principal components $\vec v_1^{\mathsf T}, \vec v_2^{\mathsf T}, \vec v_3^{\mathsf T}, \ldots$. 

Similarly, in vol surface MPCA, we put together all daily vol surface movement matrices from the past 3 months to form a 3-dimensional data cube $Z$. We then perform the [higher-order singular value decomposition (HOSVD)](https://en.wikipedia.org/wiki/Higher-order_singular_value_decomposition) which will give us two unitary matrices $A$ and $B$, and a 3-dimensional score cube $U$, such that any specific layer of the score cube $U^{(k)}$ and its corresponding vol surface movement matrix $Z^{(k)}$ (from a specific date) satisfy the identity 
$$
Z^{(k)} = AU^{(k)}B^{\mathsf{T}}.
$$
The unitary matrices $A$ and $B$ together play the same role as $V$ in SVD. Denote the column vectors of $A$ by $\vec a_1, \vec a_2, \ldots$, and the column vectors of $B$ by $\vec b_1, \vec b_2, \ldots$. The multilinear principal component surfaces are the [outer product](https://en.wikipedia.org/wiki/Outer_product) $\vec a_i\otimes \vec b_j$. 


## Application in Portfolio Hedging

In curve PCA, we write tomorrow's yield curve movement as 
$$
\vec x \approx t_1\vec v_1 + t_2\vec v_2 + t_3\vec v_3, 
$$
where $t_1, t_2$ and $t_3$ are random variables with variances estimated from the score matrix $T$. We further write tomorrow's PnL attribution as 
$$
\langle\vec x, \vec d\rangle \approx t_1\langle \vec v_1, \vec d\rangle + t_2\langle \vec v_2, \vec d\rangle + t_3\langle \vec v_3, \vec d\rangle, 
$$ 
where $\vec d$ is today's EOD portfolio delta ladder. The 3 terms on the right-hand side correspond to PnL contributed by PC1, PC2 and PC3, respectively. Now, to hedge out PC1 risk (that's PnL contributed by parallel shift of the curve), we add a curve instrument to the portfolio, changing the PnL to 
$$
\langle\vec x, \vec d+h \vec b\rangle \approx t_1\langle \vec v_1, \vec d+\beta \vec h\rangle + t_2\langle \vec v_2, \vec d+\beta \vec h\rangle + t_3\langle \vec v_3, \vec d+\beta \vec h\rangle, 
$$ 
where $\vec h$ is the delta ladder of the curve instrument added, and $\beta$ the hedge ratio. To make the portfolio immune to parallel shift of the curve, we just have to find the hedge ratio $\beta$ such that $\langle \vec v_1, \vec d+\beta \vec h\rangle = 0$. The hedge ratio is 
$$
\beta = -\frac{\langle \vec v_1, \vec d\rangle }{\langle \vec v_1, \vec h\rangle }. 
$$


Similarly, in vol surface MPCA, we write tomorrow's vol surface movement as 
$$
Z \approx \sum_{i=1}^2\sum_{j=1}^2 u_{ij}(\vec a_i\otimes\vec b_j), 
$$
where $u_{ij}$'s are random variables with variances estimated from the score cube $U$. Empirically, on the USD swaption vol surface, the 4 terms combined can often explain over $90\%$ of the variances of the data. Given today's EOD vega matrix $D$, same size as $Z$, tomorrow's PnL is sum of elementwise products of $D$ and $Z$, which is conveniently given by the [trace](https://en.wikipedia.org/wiki/Trace_(linear_algebra)) of $ZD^{\mathsf T}$, which we denote by $\text{tr}(ZD^{\mathsf T})$. It can be shown that 
\begin{align*}
\text{tr}(ZD^{\mathsf T}) &\approx \text{tr}\left(\left(\sum_{i=1}^2\sum_{j=1}^2 u_{ij}(\vec a_i\otimes\vec b_j)\right)D^{\mathsf T}\right)\\
&= \sum_{i=1}^2\sum_{j=1}^2 u_{ij}(\vec a_i^{\mathsf T} D \vec b_j). 
\end{align*}
This follows from the property of trace and outer product. To hedge out MPC1 risk, we add one instrument to the portfolio, say a 1y10y swaption with notional $\beta$, whose vega matrix is $\beta H$, changing the PnL to 
\begin{align*}
\text{tr}(Z(D+\beta H)^{\mathsf T}) = \sum_{i=1}^2\sum_{j=1}^2 u_{ij}(\vec a_i^{\mathsf T} (D+\beta H) \vec b_j). 
\end{align*}
To find the hedge ratio that makes the MPC1 term vanish, we set $\vec a_1^{\mathsf T} (D+\beta H) \vec b_1 = 0$, which solves to
$$
\beta = -\frac{(\vec a_1^{\mathsf T} D \vec b_1)}{(\vec a_1^{\mathsf T} H \vec b_1)}. 
$$
